# Cопоставление геоназваний с унифицированными именами geonames

# Задача

- Создать решение для подбора наиболее подходящих названий с geonames.
Например Ереван -> Yerevan
- На примере РФ и стран наиболее популярных для релокации - Беларусь, Армения,
Казахстан, Кыргызстан, Турция, Сербия. Города с населением от 15000 человек (с
возможностью масштабирования на сервере заказчика)
- Возвращаемые поля *geonameid, name, region, country, cosine similarity*
- формат данных на выходе: список словарей, например [{dict_1}, {dict_2}, …. {dict_n}], где словарь - одна запись с указанными полями

# Цель

Сопоставление произвольных гео названий с унифицированными именами geonames для внутреннего использования Карьерным центром.

# План работы

- Провеcти исследовательский анализ данных.
- Подготовить данные к обучению.
  - Первый способ через подключение сторонних сайтов с помощью API
  - Второй способ обучение модели с помощью DML.
    - Обучить нейронную сеть и рассчитайте её качество.

# Описание данных

Используемые таблицы с geonames:
- admin1CodesASCII
- alternateNamesV2
- cities15000
- countryInfo
- при необходимости любые другие открытые данные
- таблицы geonames можно скачать здесь http://download.geonames.org/export/dump/
- Тестовый датасет: https://disk.yandex.ru/d/wC296Rj3Yso2AQ


In [1]:
%load_ext autoreload
%autoreload 2

**Импортируем и устаналиваем необходимые библиотеки (pandas, numpy, transformers и другие).**

In [2]:
!pip install pyspellchecker

In [3]:
!pip install -U sentence_transformers

from sentence_transformers import SentenceTransformer, InputExample, util

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/mbart-aug/corpus.csv
/kaggle/input/mbart-aug/augmentation_add_typo.py
/kaggle/input/mbart-aug/geo_test.csv
/kaggle/input/mbart-aug/city_dataset.py


In [5]:
import sys
sys.path.append('/kaggle/input/mbart-aug/')

In [6]:
import os
import re
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from augmentation_add_typo import AugTypo
from tqdm.auto import tqdm
from tqdm import notebook

# NLP

import torch
import torch.nn as nn
import transformers

from transformers import (MBartForConditionalGeneration,
                          MBart50TokenizerFast,
                          AdamW,
                          get_linear_schedule_with_warmup,
                          DataCollatorWithPadding)
from torch.utils.data import Dataset, DataLoader, TensorDataset
from city_dataset import CityDataset

import nltk

Так как расчеты проводятся в Google Colab. То сразу загрузим запрос из базы данных в виде таблицы **corpus_rus**.

In [7]:

corpus_rus = pd.read_csv("/kaggle/input/mbart-aug/corpus.csv", delimiter=';')

In [ ]:
if os.path.exists('corpus.csv'):
    corpus_rus = pd.read_csv("corpus.csv", delimiter=';')
elif os.path.exists('C:/Users/User/Desktop/DS Python/Geonames/data/corpus.csv'):
    corpus_rus = pd.read_csv("C:/Users/User/Desktop/DS Python/Geonames/data/corpus.csv",
                           delimiter=';')
else:
    print('Something is wrong')

corpus_rus.head()

Проверим, какие страны у нас были в запросе и таблице.

In [8]:
corpus_rus['country'].unique()

array(['Armenia', 'Belarus', 'Kyrgyzstan', 'Kazakhstan', 'Russia',
       'Serbia', 'Turkey'], dtype=object)

**Подготовим данные к обучению.**

In [9]:
# Удаляем пропущенные значения из таблицы
corpus = corpus_rus.dropna()
# Разделяем значения в столбце 'alternatenames'
corpus.alternatenames = corpus.alternatenames.str.split(',')
# Приводим к виду 1 наименование asciiname = 1 наименование alternatenames
corpus = corpus.explode('alternatenames')
# Удаляем совпадающие значения в столбцах
corpus = corpus[corpus.asciiname!=corpus.alternatenames]
# Удаляем парные дубликаты из двух столбцов
corpus = corpus.drop_duplicates(subset=['asciiname', 'alternatenames'])

/tmp/ipykernel_6976/3393550930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus.alternatenames = corpus.alternatenames.str.split(',')


**Выведем первые 5 строк преобразованной таблицы` и размер на экран.**

In [10]:
corpus.head()

,geonameid,name,asciiname,alternatenames,population,country,country_code,admin1_code,concat_code,region,ascii_region
0,174875,Kapan,Kapan,Ghap'an,33160,Armenia,AM,08,AM.08,Syunik,Syunik
0,174875,Kapan,Kapan,Ghapan,33160,Armenia,AM,08,AM.08,Syunik,Syunik
0,174875,Kapan,Kapan,Ghap’an,33160,Armenia,AM,08,AM.08,Syunik,Syunik
0,174875,Kapan,Kapan,Kafan,33160,Armenia,AM,08,AM.08,Syunik,Syunik
0,174875,Kapan,Kapan,Kafin,33160,Armenia,AM,08,AM.08,Syunik,Syunik


In [11]:
corpus.shape

(22722, 11)

Список оригинальных названий городов, для которых будем искать совпадения.

In [ ]:
name = corpus.asciiname.drop_duplicates().values
name

Создадим копию нового списка, где у нас будут только необходимые данные для обучения и создания аугментаций.

In [12]:
# Удаляем пропущенные значения из таблицы
city = corpus_rus.dropna()
# Разделяем значения в столбце 'alternatenames'
city.alternatenames = city.alternatenames.str.split(',')
# Приводим к виду 1 наименование asciiname = 1 наименование alternatenames
city = city.explode('alternatenames')
# Оставляем только 3 столбца
city = city[['asciiname', 'alternatenames', 'country']]
# Удаляем парные дубликаты из двух столбцов
city = city.drop_duplicates(subset=['asciiname', 'alternatenames'])
city

/tmp/ipykernel_6976/3558713722.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city.alternatenames = city.alternatenames.str.split(',')


,asciiname,alternatenames,country
0,Kapan,Ghap'an,Armenia
0,Kapan,Ghapan,Armenia
0,Kapan,Ghap’an,Armenia
0,Kapan,Kafan,Armenia
0,Kapan,Kafin,Armenia
...,...,...,...
1701,Beylikduezue,Beylikduzu,Turkey
1703,Cankaya,Cankaya,Turkey
1703,Cankaya,Çankaya,Turkey
1709,Muratpasa,Muratpasa,Turkey


Создадим экземпляр класса AugTypo()

In [13]:
aug_typo = AugTypo()

Создадим новые имена из альтернативных, внеся опечатку

In [14]:
# Создаем аугментированную таблицу с опечатками
augmented_names = []
for i in range(len(list(city['alternatenames']))):
    augmented_names.append(aug_typo.add_typo(list(city['alternatenames'])[i],
                                             list(city['country'])[i]))

In [15]:
print('Длина нового датасета с опечатками:', len(augmented_names))

Длина нового датасета с опечатками: 24338


In [16]:
print('Оценим несколько новых имен:', augmented_names[:7])

Оценим несколько новых имен: ["Ghap'էn", 'Ghaձan', 'Ghզp’an', 'Kaէan', 'Kafiթ', 'Kճpan', 'Kapanջs']


Загрузим модель из библиотеки transformers.

In [17]:
# Загрузка предварительно обученной модели mBART и токенизатора
checkpoint = 'facebook/mbart-large-50-many-to-many-mmt'
tokenizer = MBart50TokenizerFast.from_pretrained(checkpoint)
mbart_model = MBartForConditionalGeneration.from_pretrained(checkpoint)

Добавим новый столбец с опечатками в таблицу city.

In [18]:
city['augmented_names'] = augmented_names
city['sum_aug'] = city['alternatenames'] + ',' + city['augmented_names']
city

,asciiname,alternatenames,country,augmented_names,sum_aug
0,Kapan,Ghap'an,Armenia,Ghap'էn,"Ghap'an,Ghap'էn"
0,Kapan,Ghapan,Armenia,Ghaձan,"Ghapan,Ghaձan"
0,Kapan,Ghap’an,Armenia,Ghզp’an,"Ghap’an,Ghզp’an"
0,Kapan,Kafan,Armenia,Kaէan,"Kafan,Kaէan"
0,Kapan,Kafin,Armenia,Kafiթ,"Kafin,Kafiթ"
...,...,...,...,...,...
1701,Beylikduezue,Beylikduzu,Turkey,Bemlikduzu,"Beylikduzu,Bemlikduzu"
1703,Cankaya,Cankaya,Turkey,Cankara,"Cankaya,Cankara"
1703,Cankaya,Çankaya,Turkey,Çankhya,"Çankaya,Çankhya"
1709,Muratpasa,Muratpasa,Turkey,turatpasa,"Muratpasa,turatpasa"


Теперь подготовим данные к обучению. Созданный столбец с суммарными названиями городов

In [19]:
# Создание списка городов и универсальных названий
city = city.dropna()
# Разделяем значения в столбце 'sum_aug'
city.sum_aug = city.sum_aug.str.split(',')
# Приводим к виду 1 наименование asciiname = 1 наименование sum_aug
city = city.explode('sum_aug')
# Удаляем совпадающие значения в столбцах
city = city[city.asciiname!=city.sum_aug]
# Удаляем парные дубликаты из двух столбцов
city = city.drop_duplicates(subset=['asciiname', 'sum_aug'])
city

,asciiname,alternatenames,country,augmented_names,sum_aug
0,Kapan,Ghap'an,Armenia,Ghap'էn,Ghap'an
0,Kapan,Ghap'an,Armenia,Ghap'էn,Ghap'էn
0,Kapan,Ghapan,Armenia,Ghaձan,Ghapan
0,Kapan,Ghapan,Armenia,Ghaձan,Ghaձan
0,Kapan,Ghap’an,Armenia,Ghզp’an,Ghap’an
...,...,...,...,...,...
1703,Cankaya,Çankaya,Turkey,Çankhya,Çankaya
1703,Cankaya,Çankaya,Turkey,Çankhya,Çankhya
1709,Muratpasa,Muratpasa,Turkey,turatpasa,turatpasa
1709,Muratpasa,Muratpaşa,Turkey,huratpaşa,Muratpaşa


Токенизируем наши значения на входе в модель и на выходе.

In [20]:
# Токенизируем выходящие значения из модели (the labels)
tokenized_labels = tokenizer(list(city['asciiname']), padding=True,
                             truncation=True, return_tensors="pt")
# Токенизируем входящие значения в модель (the input sentences)
tokenized_inputs = tokenizer(list(city['sum_aug']), padding=True,
                             truncation=True, return_tensors="pt")

# Преобразуем значения в понятный для модели вид
# input_ids и attention_mask берем из the tokenized input
input_ids = tokenized_inputs["input_ids"]
attention_mask = tokenized_inputs["attention_mask"]
labels = tokenized_labels["input_ids"]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Разделим данные на тренировочную и валидационную выборки

In [21]:
input_ids_train, input_ids_val, attention_mask_train, attention_mask_val, labels_train, labels_val = \
    train_test_split(input_ids, attention_mask, labels, test_size=0.1, random_state=42)

Теперь создадим датасет с помощью кастомного класса CityDataset(). И передадим все в data_collator, чтобы распределить данные по батчам.

In [22]:
# Create training and validation datasets from the split
# создадим экземляр класса CityDataset
city_dataset = CityDataset(input_ids, attention_mask, labels)

# Функция Data collator
def data_collator(features):
    batch = {}
    batch['input_ids'] = torch.stack([f['input_ids'] for f in features])
    batch['attention_mask'] = torch.stack([f['attention_mask'] for f in features])
    batch['labels'] = torch.stack([f['labels'] for f in features])
    return batch

train_dataset = CityDataset(input_ids_train, attention_mask_train, labels_train)
val_dataset = CityDataset(input_ids_val, attention_mask_val, labels_val)

In [23]:
# Additional setup for validation, loss tracking, and consistent device usage
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(val_dataset, batch_size=16, collate_fn=data_collator)


Определим оптимизатор

In [24]:
# Определение оптимизатора и настройка параметров обучения
optimizer = AdamW(mbart_model.parameters(), lr=5e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Определим количество эпох

In [25]:
# Количество эпох и шагов для обучения
num_epochs = 8
num_training_steps = len(train_dataloader) * num_epochs

Настроим параметры обучения

In [26]:
# настройка параметров обучения
scheduler = get_linear_schedule_with_warmup(optimizer,
                                        num_warmup_steps=0,
                                        num_training_steps=num_training_steps)

Определим устройство GPU и передадим ему модель.

In [27]:
# использовать GPU в случае, если у нас будет такая возможность
device = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")

mbart_model.to(device)

# Если есть несколько GPU

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    mbart_model = nn.DataParallel(mbart_model, device_ids=[1, 0])

Let's use 2 GPUs!


Теперь начнем обучение

In [28]:
# Цикл обучения
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    mbart_model.train()
    
    epoch_losses = []
    epoch_numbers = []
    
    for batch in train_dataloader:

        # Распаковка данных из batch
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Проход вперед (Forward pass)
        outputs = mbart_model(input_ids=input_ids, 
                              attention_mask=attention_mask,
                              labels=labels)
        loss = outputs.loss
        loss = loss.mean()
        
        # Обратный проход (Backward pass)
        loss.backward()

        # Обновление параметров модели
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
                
        epoch_losses.append(loss.item())
        progress_bar.update(1)
    
    mbart_model.eval()  # Set the model to evaluation mode
    val_losses = []
    for batch in val_dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
        
            # Forward pass for validation
            outputs = mbart_model(input_ids=input_ids, 
                                  attention_mask=attention_mask, 
                                  labels=labels)
            val_loss = outputs.loss
            val_loss = val_loss.mean()
            val_losses.append(val_loss.item())

        progress_bar.update(1)
        
    # Calculate and print average validation loss for this epoch
    avg_train_loss = sum(epoch_losses) / len(epoch_losses)
    print(f'Train Loss for epoch {epoch + 1}: {avg_train_loss}')    
        
    # Calculate and print average validation loss for this epoch
    avg_val_loss = sum(val_losses) / len(val_losses)
    print(f'Validation Loss for epoch {epoch + 1}: {avg_val_loss}')

  0%|          | 0/21176 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Train Loss for epoch 1: 0.3216053770667186
Validation Loss for epoch 1: 0.10830513246648972
Train Loss for epoch 2: 0.05764568902163872
Validation Loss for epoch 2: 0.07335579529641664
Train Loss for epoch 3: 0.020760579741589215
Validation Loss for epoch 3: 0.06323992654734727
Train Loss for epoch 4: 0.009985749857983823
Validation Loss for epoch 4: 0.06488018874449569
Train Loss for epoch 5: 0.005755285534919733
Validation Loss for epoch 5: 0.05692425142449068
Train Loss for epoch 6: 0.003433150064887099
Validation Loss for epoch 6: 0.04988281113215265
Train Loss for epoch 7: 0.0017402949327766357
Validation Loss for epoch 7: 0.04935292155528028
Train Loss for epoch 8: 0.0010005620061387579
Validation Loss for epoch 8: 0.048332689799174704


Сохраним модель и токенизатор, а также все необходимые файлы.

In [30]:
# Сохранение дообученной модели
output_model_path = "/kaggle/working/mbart_aug"
mbart_model.module.save_pretrained(output_model_path)
tokenizer.save_pretrained(output_model_path)
# Сохранение файла pytorch_model.bin
state_dict = mbart_model.state_dict()
torch.save(state_dict, "/kaggle/working/mbart_aug/pytorch_model.bin")
tokenizer.save_pretrained(output_model_path)

('/kaggle/working/mbart_aug/tokenizer_config.json',
 '/kaggle/working/mbart_aug/special_tokens_map.json',
 '/kaggle/working/mbart_aug/sentencepiece.bpe.model',
 '/kaggle/working/mbart_aug/added_tokens.json',
 '/kaggle/working/mbart_aug/tokenizer.json')

## Анализ обученной модели

**Анализ выборки:** 

Выборка альтернативных имен имеен различные виды написания оригинальных названий городов. Количество оригинальных написаний для каждого города разное. Но также некоторые написания городов отсутствуют, например, "екб" - для Екатеринбурга.

**Анализ модели:** Модель обучена на 8 эпохах с использованием аугментации для каждого слова из альтернативных имен. (была создана одна случайная опечатка).

При оценке качества модели тем же способом, что и в файле `geonames _test_file`, показала результаты `'score' = 0.8276` чуть хуже, чем модель на 4 эпохах с аугментациями `'score' = 0.8448` (которая и была добавлена на hugginFace и использовалась для демонстрации результатов на тестовой выборке). 
Но при этом, можно заметить, что функция Loss продолжает падать и не вышло в область 'плато'. 

```
Train Loss for epoch 1: 0.3216053770667186
Validation Loss for epoch 1: 0.10830513246648972
Train Loss for epoch 2: 0.05764568902163872
Validation Loss for epoch 2: 0.07335579529641664
Train Loss for epoch 3: 0.020760579741589215
Validation Loss for epoch 3: 0.06323992654734727
Train Loss for epoch 4: 0.009985749857983823
Validation Loss for epoch 4: 0.06488018874449569
Train Loss for epoch 5: 0.005755285534919733
Validation Loss for epoch 5: 0.05692425142449068
Train Loss for epoch 6: 0.003433150064887099
Validation Loss for epoch 6: 0.04988281113215265
Train Loss for epoch 7: 0.0017402949327766357
Validation Loss for epoch 7: 0.04935292155528028
Train Loss for epoch 8: 0.0010005620061387579
Validation Loss for epoch 8: 0.048332689799174704
```
То есть, чтобы добиться лучших показателей от модели, нужно рассчитать большее количество эпох и по Loss функции выбрать наиболее оптимальную. 

Так как нет мощностей для расчета на большее количество эпох, данный вопрос оставляю открытым.

Как можно было бы улучишь качество:

- Качество данных на входе: оценить еще раз те слова, которые модель не смогла предугадать, и добавить дополнительные аугментации.
- Анализ ошибок: к примеру модель сильно ошибается, когда названия городов исковерканы значительно, то есть более 3-4 ошибок в слове, зато почти идеально работает с меньшим количеством ошибок в слове.

В целом, нужно понимать, что у слова есть некий порог узнаваемости, то есть существует приемлемое количество ошибок, которые ты можешь совершить для того, чтобы модель (а может и человек) поняла какой город имеется ввиду. Например, для слова из 4 букв - 2 ошибки достаточно критичны, то есть слово достаточно сильно изменено.